In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [3]:
def sort_contours(cnts, method="left-to-right"):
    # initialize the reverse flag and sort index
    reverse = False
    i = 0
    
    # handle if we need to sort in reverse
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True
        
    # handle if we are sorting against the y-coordinate rather than
    # the x-coordinate of the bounding box
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1
        
    # construct the list of bounding boxes and sort them from top to
    # bottom
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
        key=lambda b:b[1][i], reverse=reverse))
    
    # return the list of sorted contours and bounding boxes
    return (cnts, boundingBoxes)


In [4]:
def get_centres(cnts):
    centres = []
    
    for i in range(len(cnts)):
        moments = cv2.moments(cnts[i])
        if moments['m10'] == 0 or moments['m00'] == 0 or moments['m01'] == 0:
            continue
            
        centres.append((int(moments['m10']/moments['m00']), 
                        int(moments['m01']/moments['m00'])))
    
    return centres

In [10]:
img = cv2.imread('canny1.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
res = cv2.resize(img, None,fx=2, fy=2, interpolation = cv2.INTER_CUBIC)
res = res[300:600][:].copy() #ROI

_, cnts, _ = cv2.findContours(res, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_TC89_KCOS)
cnts, _ = sort_contours(cnts, method="left-to-right")
cnts = [cv2.approxPolyDP(cnt, 4, True) for cnt in cnts]
centres = get_centres(cnts)

res = cv2.cvtColor(res, cv2.COLOR_GRAY2BGR)

for idx in range(1, len(centres),2):
    length = np.array(centres[idx]) - np.array(centres[idx-1])
    
    if length[0] < 0:
        continue
    
    cv2.circle(res, centres[idx-1], 1, (0,255,0), 5)
    cv2.circle(res, centres[idx], 1, (0,255,0), 5)
    cv2.line(res, centres[idx-1], centres[idx], (0,255,0), 1)
    
    point = centres[idx-1]
    cv2.putText(res, str(length[0]*0.008)+"cm", tuple([point[0], point[1]+50]), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255), thickness=1)


(480, 640, 3)
(480, 640)
(300, 1280)


In [6]:
cv2.imwrite('contours.png', res)

True